### **Import Packages**

In [ ]:
import os
from os import path
from typing import Optional
import numpy as np
import pandas as pd
import glob
from tqdm.auto import tqdm
import cv2 as cv
import matplotlib.pyplot as plt
import sys
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
from typing import Optional
import pyopencl as cl
import tensorflow as tf 
from rocm.configure import * 
from IPython.display import HTML
from base64 import b64encode
import keras
import getpass
from imutils import paths
import imageio
from IPython.display import Image

### **Constants**

In [ ]:
import json
with open('./configs/data_path.json', 'r') as f:
    data = json.load(f)

USER = getpass.getuser()

# Large Dataset
DATA_FOLDER = data['dataset'][USER][0]
COMPRESSED_DATA_FOLDER = data['dataset'][USER][1]
DATA_DIRECTORIES = os.listdir(DATA_FOLDER)

# Define the data directories
TEST_FOLDER = data['test']
TRAIN_FOLDER = data['train']
VAL_FOLDER = data['val']

# Define hyperparameters
IMG_SIZE = data['image_size']
BATCH_SIZE = data['batch_size']
EPOCHS = data['epochs']
MAX_SEQ_LENGTH = data['max_seq_length']
NUM_FEATURES = data['num_features']

### **Utility Functions**

In [ ]:
# Function read video metadata from json files
def read_meta_from_json(data_directories, data_folder, processed_data_folder):
    
    meta_df = pd.DataFrame()

    for index, part_folder in enumerate(data_directories):
        part_path = os.path.join(data_folder, part_folder)
        json_file = next(file for file in os.listdir(part_path) if file.endswith('json'))
        json_file_path = os.path.join(part_path, json_file)
        
        part_df = pd.read_json(json_file_path).T
        part_df['part'] = index
        part_df['path'] = part_path
        part_df['path-compressed'] =  os.path.join(processed_data_folder, os.path.basename(part_path))
        part_df['filename'] = part_df.index
        
        meta_df = pd.concat([meta_df, part_df])

    # Display 5 random rows from meta_df
    meta_df.reset_index(drop=True, inplace=True)
    display(meta_df.head(n=5))
    return meta_df    

# Function to check data types
def explore_files(files):
    ext_dict = []
    for file in files:
        file_ext = file.split('.')[1]
        if (file_ext not in ext_dict):
            ext_dict.append(file_ext) 

    for file_ext in ext_dict:
        print(f"Files with extension `{file_ext}`: {len([file for file in files if  file.endswith(file_ext)])}")


### **Step 0: Define the Device (GPU)**

In [ ]:
device = tf.config.list_physical_devices('GPU')
print(f'Using device:\n{device[0][0]} for training with tensorflow {tf.__version__}')
print(f"Num GPUs Available: {len(tf.config.experimental.list_physical_devices('GPU'))}")
print(f"Num CPUs Available: {len(tf.config.list_physical_devices('CPU'))}")

### **Set GPU Memory Growth**
By default, TensorFlow allocates all GPU memory when it starts. If you want TensorFlow to allocate memory only as needed, you can set GPU memory growth:

In [ ]:
if gpus:= tf.config.experimental.list_physical_devices('GPU'):
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        
        # Allow TensorFlow to see only one GPU
        tf.config.set_visible_devices(gpus[0], 'GPU')
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

### **Step 1: Define the Model**

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models  # type: ignore

def build_3d_cnn_model(input_shape):
    return models.Sequential()

model = build_3d_cnn_model((32, 32, 32, 1))

### **Step 2: Define the Dataset and DataLoader**

The data is comprised of .mp4 files, split into compressed sets of ~10GB apiece. A metadata.json accompanies each set of .mp4 files, and contains filename, label (REAL/FAKE), original and split columns, listed here:
- ```filename```  -  the filename of the video
- ```label```  -  whether the video is REAL or FAKE
- ```original```  -  in the case that a train set video is FAKE, the original video is listed here
- ```split```  -  this is always equal to "train".

#### What are we predicting?
We predict whether or not a particular video is a deepfake. A deepfake could be **either** a face or voice swap (or both). In the training data, this is denoted by the string ```"REAL"``` or ```"FAKE"``` in the label column. In your submission, you will predict the probability that the video is a fake. The full training set is just over 470 GB, divided into 50 smaller files, each ~10 GB in size. We decided to split this data according to the following ratio:

**Training Set** - ``80%`` | **Validation Set** - ``10%`` | **Test Set** - ``10%``


1. Read `metadata.json`files from each folder
2. Load the data
3. Split the data (train, validation, and test)
4. Process and save the data
5. Load the processed data

In [ ]:
# Read all json metadata files into a dataframe
meta_df = read_meta_from_json(DATA_DIRECTORIES, DATA_FOLDER, COMPRESSED_DATA_FOLDER)

In [ ]:
def preprocess_video(video_path):
    video = tf.io.read_file(video_path)
    video = tf.io.decode_video(video)
    video = tf.image.resize(video, (224, 224), method='bilinear', preserve_aspect_ratio=True)
    video = tf.cast(video, tf.float32) / 255.0  # Normalize to [0, 1]
    return video

def load_video_dataset(file_paths, labels):
    dataset = tf.data.Dataset.from_tensor_slices((file_paths, labels))
    dataset = dataset.map(lambda x, y: (preprocess_video(x), y))
    dataset = dataset.batch(8)
    return dataset

print(f"Train samples: {len(os.listdir(os.path.join(DATA_FOLDER, TRAIN_FOLDER)))}")
print(f"Test samples: {len(os.listdir(os.path.join(DATA_FOLDER, TEST_FOLDER)))}")

# Example file paths and labels
# train_file_paths = ["path/to/video1.mp4", "path/to/video2.mp4", ...]
# train_labels = [0, 1, ...]  # Corresponding labels

# train_dataset = load_video_dataset(train_file_paths, train_labels)

### **Step 3: Define the Loss Function and Metrics (Score)**

In [ ]:
from keras import backend as K

def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return true_positives / (possible_positives + K.epsilon())


def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    return true_positives / (predicted_positives + K.epsilon())

def specificity(y_true, y_pred):
    tn = K.sum(K.round(K.clip((1 - y_true) * (1 - y_pred), 0, 1)))
    fp = K.sum(K.round(K.clip((1 - y_true) * y_pred, 0, 1)))
    return tn / (tn + fp + K.epsilon())

def f1(y_true, y_pred):
    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    return 2 * ((p * r) / (p + r + K.epsilon()))

def matthews_correlation_coefficient(y_true, y_pred):
    tp = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    tn = K.sum(K.round(K.clip((1 - y_true) * (1 - y_pred), 0, 1)))
    fp = K.sum(K.round(K.clip((1 - y_true) * y_pred, 0, 1)))
    fn = K.sum(K.round(K.clip(y_true * (1 - y_pred), 0, 1)))

    num = tp * tn - fp * fn
    den = (tp + fp) * (tp + fn) * (tn + fp) * (tn + fn)
    return num / K.sqrt(den + K.epsilon())

### **Step 4: Train the Model**

In [ ]:
# Assuming `model` and `train_dataset` are defined as before
model.compile(optimizer='adam', loss='binary_focal_crossentropy', metrics=['accuracy', recall, precision, specificity, f1, matthews_correlation_coefficient])

# Train the model on GPU
with tf.device('/gpu:0'):
    model.fit(train_dataset, epochs=10, validation_data=val_dataset)

# Evaluate the model
loss, accuracy = model.evaluate(val_dataset)
print(f'Validation Loss: {loss}, Validation Accuracy: {accuracy}')

### **Step 5: Plot Training Process**
